In [ ]:
import numpy as np

In [ ]:
class MulLayer:
    def __init__(self):
        self.x = None;
        self.y = None;
        
    def forward(self,x,y):
        self.x = x;
        self.y = y;
        out = x*y;
        return out;
    
    def backward(self,dout):
        dx = self.y * dout;
        dy = self.x *dout;
        return dx,dy

In [ ]:
apple=100
apple_num = 2
tax =1.1

In [ ]:
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [ ]:
apple_price = mul_apple_layer.forward(apple,apple_num)

In [ ]:
price = mul_tax_layer.forward(apple_price,tax)

In [ ]:
dprice =1 ;
dapple_price , dtax = mul_tax_layer.backward(dprice)

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None;
        
    def forward(self,x):
        self.mask = (x <= 0);
        out = x.copy();
        out[self.mask] = 0;
        return out
    
    def backward(self,dout):
        dout[self] = 0;
        dx = out;
        return dx

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None;
        
    def forward(self,x):
        self.out = 1/(1+np.exp(-x));
        
        return self.out
    
    def bacward(self,dout):
        dx = dout * (1-self.out)*self.out
        return dx

In [ ]:
sig = Sigmoid()

In [ ]:
sig.forward(np.array([1,2,3]))

In [ ]:
sig.bacward(1)

In [ ]:
class Affine:
    def __init__(self,W,b):
        self.W = W;
        self.b = b;
        self.x = None;
        self.dW = None;
        self.db = None;
        
    def forward(self,x):
        self.x = x;
        out = np.dot(x,self.W) + self.b;
        return out
        
    def backward(self,dout):
        dx = np.dot(dout,self.W.T);
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis = 0)
        return dx
        

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss= None;
        self.y= None;
        self.t = None;
        
    def forward(self,x,t):
        self.t = t;
        self.y = softmax(x);
        self.loss = cross_entropy_error(self.y,self.t);
        
        return self.loss
    
    def backward(self,dout = 1):
        batch_size = self.t.shape[0];
        
        dx= (self.y - self.t)/batch_size;
        return dx

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [20]:
del TwoLayerNet

In [2]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 生成层
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [ ]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,out_size,weight_size_std=0.01):
        self.params = {};
        self.params['W1'] =weight_size_std * np.random.randn(input_size,hidden_size);
        self.params['b1'] = np.zeros(hidden_size);
        self.params['W2'] =weight_size_std * np.random.randn(hidden_size,out_size);
        self.params['b2'] = np.zeros(hidden_size);
        
        self.layers = OrderedDict();
        self.layers["Affine1"] = Affine(self.params['W1'],self.params['b1']);
        self.layers["Relu1"] = Relu();
        self.layers["Affine2"] = Affine(self.params['W2'],self.params['b2']);
        self.lastLayer = SoftmaxWithLoss();
        
        
    def predict(self,x):
        for layer in self.layers.values():
            out = layer.forward(x)
        return out
    
    def loss(self,x,t):
        y = self.predict(x);
        out = self.lastLayer.forward(y);
        
        return out
    
    
        
        
        


In [ ]:
del Affine

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 权重和偏置参数的导数
        self.dW = None
        self.db = None

    def forward(self, x):
        # 对应张量
        
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        if self.x.ndim ==1:
            x = self.x.reshape(1,self.x.size)
            
        if dout.ndim == 1:
            dout = dout.reshape(1,dout.size)
            
        
        dx = np.dot(dout, self.W.T).reshape(self.x.shape)
        self.dW = np.dot(x.T, dout)
        self.db = np.sum(dout, axis=0)
          # 还原输入数据的形状（对应张量）
        return dx

In [ ]:
del testpara,test

In [ ]:
testparaW = np.random.randn(3,4)
testparab = np.zeros(4)

In [ ]:
testAffine = Affine(testparaW,testparab)

In [ ]:
xtest.shape

In [ ]:
xtest = np.random.randn(3)
ttest = np.random.randn(4)

In [ ]:
testAffine.forward(xtest)

In [ ]:
testAffine.backward(ttest)

In [ ]:
testAffine.dW

In [ ]:
test.predict(xtest)

In [3]:
testnetwork = TwoLayerNet(3,4,3)

In [16]:
xtest = np.random.randn(1,3)
ttest = np.random.randn(1,3)
douttest = np.random.randn(1,3)

In [17]:
testnetwork.predict(xtest)

array([[-0.00056795,  0.00022167,  0.00075566]])

In [18]:
testnetwork.loss(xtest,ttest)

1.0979929353321451

In [19]:
np.argmax(xtest,axis=1)

array([2], dtype=int64)